In [1]:
import pandas as pd
import streamlit as st
import plotly_express as px



In [2]:
car_data = pd.read_csv('vehicles_us.csv')

In [3]:
display(car_data.head(10))

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
5,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15
6,12990,2015.0,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73
7,15990,2013.0,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68
8,11500,2012.0,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,NaN,1.0,2018-07-16,19
9,9200,2008.0,honda pilot,excellent,NaN,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17


Removing all of the columns that I will not be utilizing for visualization in the web app

In [4]:
comp_df = car_data.drop(['cylinders', 'fuel', 'transmission', 'paint_color', 'is_4wd', 'date_posted', 'days_listed' ], axis=1)

Cleaning the remaining data

In [5]:
display(comp_df)
display(comp_df.info())


,price,model_year,model,condition,odometer,type
0,9400,2011.0,bmw x5,good,145000.0,SUV
1,25500,NaN,ford f-150,good,88705.0,pickup
2,5500,2013.0,hyundai sonata,like new,110000.0,sedan
3,1500,2003.0,ford f-150,fair,NaN,pickup
4,14900,2017.0,chrysler 200,excellent,80903.0,sedan
...,...,...,...,...,...,...
51520,9249,2013.0,nissan maxima,like new,88136.0,sedan
51521,2700,2002.0,honda civic,salvage,181500.0,sedan
51522,3950,2009.0,hyundai sonata,excellent,128000.0,sedan
51523,7455,2013.0,toyota corolla,good,139573.0,sedan


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   price       51525 non-null  int64  
 1   model_year  47906 non-null  float64
 2   model       51525 non-null  object 
 3   condition   51525 non-null  object 
 4   odometer    43633 non-null  float64
 5   type        51525 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 2.4+ MB


None

Checking the model year column

In [6]:
display(comp_df['model_year'].value_counts(dropna=False))

NaN       3619
2013.0    3549
2012.0    3468
2014.0    3448
2011.0    3375
          ... 
1948.0       1
1961.0       1
1936.0       1
1949.0       1
1929.0       1
Name: model_year, Length: 69, dtype: int64

Checking the Odometer column

In [7]:
display(comp_df['odometer'].value_counts(dropna=False))

NaN         7892
0.0          185
140000.0     183
120000.0     179
130000.0     178
            ... 
87836.0        1
172625.0       1
103597.0       1
167239.0       1
139573.0       1
Name: odometer, Length: 17763, dtype: int64

It looks like there is no reason for why there are null values of car year or milage other than it wasn't present to be used. With the size of the data losing these (maximum of) 11,511 entries will still leave us with over 40,000 entries. For the purposes of our visualization, I find dropping this portion of the data this to be acceptable.

In [8]:
comp_df = comp_df.dropna()
display(comp_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40563 entries, 0 to 51523
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   price       40563 non-null  int64  
 1   model_year  40563 non-null  float64
 2   model       40563 non-null  object 
 3   condition   40563 non-null  object 
 4   odometer    40563 non-null  float64
 5   type        40563 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 2.2+ MB


None

Now that we have our data cleaned of null values, I will explore the column values more and also extract useful informaiton from the columns that exist.

In [9]:
display(comp_df['model_year'].value_counts())
display(comp_df['model'].value_counts())
display(comp_df['condition'].value_counts())
display(comp_df['odometer'].value_counts())
display(comp_df['type'].value_counts())


2013.0    2976
2012.0    2932
2014.0    2906
2011.0    2873
2015.0    2809
          ... 
1948.0       1
1908.0       1
1961.0       1
1936.0       1
1949.0       1
Name: model_year, Length: 67, dtype: int64

ford f-150                  2217
chevrolet silverado 1500    1713
ram 1500                    1374
chevrolet silverado          997
jeep wrangler                882
                            ... 
acura tl                     190
chrysler 200                 185
nissan murano                185
ford f-250 super duty        179
kia sorento                  179
Name: model, Length: 99, dtype: int64

excellent    19514
good         15847
like new      3730
fair          1270
new            115
salvage         87
Name: condition, dtype: int64

0.0         172
140000.0    169
130000.0    162
120000.0    160
200000.0    153
           ... 
49478.0       1
168235.0      1
62727.0       1
91490.0       1
139573.0      1
Name: odometer, Length: 16900, dtype: int64

truck          9778
SUV            9724
sedan          9535
pickup         5477
coupe          1839
wagon          1214
mini-van        925
hatchback       842
van             468
convertible     357
other           203
offroad         178
bus              23
Name: type, dtype: int64

In the above, I notice multiple things. First there is no reason the model year should be a float value. I will chage it to an into. The model is not going to be important in my web app. So I will just extract the make from the model's name to make a new column. The condition column looks good to me--as does the odometer and type.

In [10]:
comp_df['model_year'] = comp_df['model_year'].astype(int)
comp_df['make'] = comp_df['model'].str.split().str[0]


In [11]:
comp_df = comp_df.drop('model', axis=1)

In [12]:
display(comp_df)

,price,model_year,condition,odometer,type,make
0,9400,2011,good,145000.0,SUV,bmw
2,5500,2013,like new,110000.0,sedan,hyundai
4,14900,2017,excellent,80903.0,sedan,chrysler
5,14990,2014,excellent,57954.0,sedan,chrysler
6,12990,2015,excellent,79212.0,sedan,toyota
...,...,...,...,...,...,...
51518,3750,2005,excellent,110200.0,sedan,ford
51520,9249,2013,like new,88136.0,sedan,nissan
51521,2700,2002,salvage,181500.0,sedan,honda
51522,3950,2009,excellent,128000.0,sedan,hyundai


Here we have all the columns I will want to have access to in my web app visualization. I have also removed all other columns to help reserve memory and improve speeds.

In [39]:
display(comp_df['odometer'].describe())

count     40563.000000
mean     115425.444099
std       65219.802462
min           0.000000
25%       70000.000000
50%      113000.000000
75%      155000.000000
max      990000.000000
Name: odometer, dtype: float64